In [2]:
# Useful imports
import numpy as np, matplotlib.pyplot as plt
import sys; sys.path.append('./')
import models
from data_loader import DataLoader
import emcee, corner
from sklearn.covariance import EmpiricalCovariance
import numpy.linalg as lin
import seaborn as sns

In [3]:
# Which data to load?
# E_min : [0.676, 0.84 , 1.269, 1.741, 2.12 , 2.609, 2.609, 3.586, 4.332, 5.475]
# E_max : [0.706, 0.868, 1.292, 1.759, 2.137, 2.624, 2.624, 3.598, 4.342, 5.484]
E_min = 0.676 # MeV
E_max = 4.342 # MeV
which_data = 'som'

# Select the parameterization
parameterization = 'init-f-wave'

# Use theory cov?
use_theory_cov = False

In [4]:
loader = DataLoader(E_min, E_max, which_data)

data = loader.get_data()
norm_group = loader.get_normalization_grouping()
gauss_prior_f = loader.get_normalization_prior_info()

In [5]:
# # # Set the parameter bounds and initialize the model
if parameterization == 'standard':
    labels = ['A0', 'r0', 'A1+', 'r1+', 'P1+', 'A1-', 'r1-', 'P1-']
    sys.stderr.write('No Longer Supported....')
    sys.exit(-1)
elif parameterization == 'bound_state':
    labels = ['A0', 'r0', 'r1+', 'P1+', 'r1-', 'P1-']
    param_bounds = np.array([[-0.02, 0.06], [-3, 3], [-1, 1], [-6, 6], [-1, 1], [-6, 6]])
    params_prior = np.array([[0.025, 0.015], [0.8, 0.4], [0.0, 0.1], [0.0, 1.6], [0.0, 0.1], [0.0, 1.6]]) # center, width
    gauss_prior_params = np.hstack([param_bounds, params_prior])
    sys.stderr.write('Not implemented yet...')
    sys.exit(-1)
elif parameterization == 'bs_C':
    labels = ['$A_{0}$', '$r_{0}$', '$(C_{1}^{+})^{2}$', '$P_{1}^{+}$', '$(C_{1}^{-})^{2}$', '$P_{1}^{-}$']
    param_bounds = np.array([[-0.02, 0.06], [-3, 3], [5.0, 25.0], [-6, 6], [5.0, 25.0], [-6, 6]])
    params_prior = np.array([[0.025, 0.015], [0.8, 0.4], [13.84, 1.63], [0.0, 1.6], [12.59, 1.85], [0.0, 1.6]]) # center, width
    gauss_prior_params = np.hstack([param_bounds, params_prior])
    model = models.BS_C(data, norm_group, gauss_prior_params, gauss_prior_f, use_theory_cov)
elif parameterization == 'init-f-wave':
    labels = ['A0', 'r0', 'C1+^2', 'P1+', 'C1-^2', 'P1-', 'r3+']
    param_bounds = np.array([[-0.02, 0.06], [-3, 3], [5.0, 25.0], [-6, 6], [5.0, 25.0], [-6, 6], [-3, 0]])
    params_prior = np.array([[0.025, 0.015], [0.8, 0.4], [13.84, 1.63], [0.0, 1.6], [12.59, 1.85], [0.0, 1.6], [-0.5, 1]]) # center, width
    gauss_prior_params = np.hstack([param_bounds, params_prior])
    model = models.F_Wave_AR(data, norm_group, gauss_prior_params, gauss_prior_f, use_theory_cov)

# Add normalization labels
for i in range(0, int(np.max(norm_group) + 1)):
    labels.append('f_{}'.format(i))

In [37]:
map_values = np.array([0.0138, 0.7338, 17.4373, 1.7953, 13.6261, 0.7669, -0.1668, 0.9710, 0.9216, 1.0572, 1.0110, 1.0313, 1.0278, 1.0291, 1.0568, 1.0568, 1.0692])
dof = np.array([17, 29, 45, 46, 52, 52, 52, 52, 53])
energies = np.array([0.706, 0.868, 1.292, 1.759, 2.137, 2.624, 2.624, 3.598, 4.342])
params = map_values[:model.erp_dim]
params_f = map_values[model.erp_dim:]

theory = model.cs_theory(params)
norm = params_f[model.norm_grouping.astype(int)]
inv_cov = model.inv_cov_matrix
res = norm * theory - model.cs_data

In [52]:
chi_squareds = np.zeros(int(np.max(norm_group) + 1))
for i in range(0, norm_group.shape[0]):
    # chi_squareds[int(norm_group[i])] += res[i] * second_piece[i]
    chi_squareds[int(norm_group[i])] += res[i]**2 * inv_cov[i, i]

In [53]:
chi_squareds_per_dof = chi_squareds / dof

In [78]:
print('   E  | chi^2 per dof')
print('--------------')
for i in range(0, chi_squareds_per_dof.shape[0]):
    print('{:.4} | {:.4}'.format(energies[i], chi_squareds_per_dof[i]))

   E  | chi^2 per dof
--------------
0.706 | 0.7046
0.868 | 1.085
1.292 | 1.92
1.759 | 2.072
2.137 | 2.443
2.624 | 2.751
2.624 | 1.739
3.598 | 3.35
4.342 | 4.019
